In [4]:
import pandas as pd
import sqlite3
import json
import re

# -----------------------------
# STEP 1: LOAD ORDERS (CSV)
# -----------------------------
orders = pd.read_csv("orders.csv")

# -----------------------------
# STEP 2: LOAD USERS (ULTRA-ROBUST)
# -----------------------------
with open("users.json", "r", encoding="utf-8", errors="ignore") as f:
    text = f.read()

# Find ALL JSON objects {...}
objects = re.findall(r'\{[^{}]*\}', text)

users_list = []
for obj in objects:
    try:
        users_list.append(json.loads(obj))
    except:
        pass

users = pd.DataFrame(users_list)

print("Users loaded:", users.shape)
print("User columns:", users.columns.tolist())

if "user_id" not in users.columns:
    raise ValueError("❌ user_id column not found in users data")

# -----------------------------
# STEP 3: LOAD RESTAURANTS (SQL)
# -----------------------------
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("restaurants.sql", "r", encoding="utf-8") as f:
    cursor.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print("Restaurants loaded:", restaurants.shape)

# -----------------------------
# STEP 4: MERGE DATASETS (LEFT JOIN)
# -----------------------------
final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

# -----------------------------
# STEP 5: EXPORT FINAL DATASET
# -----------------------------
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

print("✅ final_food_delivery_dataset.csv created successfully")



Users loaded: (2386, 4)
User columns: ['user_id', 'name', 'city', 'membership']
Restaurants loaded: (500, 4)
✅ final_food_delivery_dataset.csv created successfully
